In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error
import os

In [2]:
exp_folder =  "/Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/automated_labeling_experiments"
gpt_folder = os.path.join(exp_folder, "gpt_experiments")
embedding_folder = os.path.join(exp_folder, "embedding_experiments")
annotations_folder = os.path.join(exp_folder, "annotations_combined")

In [3]:
# combine all annotations of V101 and V200 into one file
# Specify the folder path containing the CSV files

for exp_id, output_folder in [["exp1", "V101.csv"], ["exp2", "V200.csv"]]:
        
    folder_paths = [os.path.join(gpt_folder, "annotations"), os.path.join(embedding_folder, "annotations")]

    csv_files = []
    # Get a list of all CSV files in the folder
    for folder_path in folder_paths:
        csv_files += [os.path.join(folder_path, file) for 
                      file in os.listdir(folder_path) if (file.endswith('.csv') & file.startswith(exp_id))]

    # Create an empty DataFrame to store the combined data
    combined_data = pd.DataFrame()

    # Iterate through each CSV file
    for file in csv_files:
        # Read the CSV file into a DataFrame
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)

        # Append the data to the combined DataFrame
        combined_data = pd.concat([combined_data, data], ignore_index=True)

    # Save the combined data to a new CSV file
    combined_data.drop_duplicates(inplace=True)
    combined_data.to_csv(os.path.join(exp_folder, "annotations_combined", output_folder), index=False)

In [4]:
def smoothness_to_int(smoothness):
    if smoothness == "very_bad":
        return 5
    elif smoothness == "bad":
        return 4
    elif smoothness == "intermediate":
        return 3
    elif smoothness == "good":
        return 2
    elif smoothness == "excellent":
        return 1
    else:
        return None

In [5]:
def exp_results(exp_id, annotations_folder, folder, class_selection, exp_type):
    ds_version = "101" if exp_id == 1 else "200"
    
    # annotataions file of entire dataset
    annot = pd.read_csv(f"{annotations_folder}/V{ds_version}.csv")

    # remove true / false classifications
    if "correct" in annot.columns:
        annot = annot[annot.correct.isna()]

    annot.loc[annot.nostreet.notna(), "surface"] = "nostreet"
    annot.loc[annot.nostreet.notna(), "smoothness"] = "nostreet"
    annot["image_id"] = annot.image.apply(lambda x: str.split(x, "/")[-1]).apply(lambda x: int(str.split(x, ".jpg")[0]))

    pred_folder = os.path.join(folder, "results")
    df = pd.DataFrame()
    counts = pd.DataFrame()
    file_name_addition = ""
    
    for cl in class_selection:
        surface = cl[0]
        smoothness = cl[1]
        batch_id = None if len(cl) <= 2 else cl[2]
        if exp_id == 1:
            if "gpt" in folder:
                results_file = f"experiment_{exp_id}_V{ds_version}_{surface}_{smoothness}.csv"
            else:
                results_file = f"V101_effnet_{surface}_{smoothness}_gpt_combined.csv"

        elif exp_id == 2:
            file_name_addition = f"_batch_{batch_id}"
            results_file = f"experiment_{exp_id}_V{ds_version}_{surface}{file_name_addition}.csv"

        results = pd.read_csv(f"{pred_folder}/{results_file}")
        
        total_n = len(results)
        pred_annot_class = results[(results.preselection_type == surface) & (results.prediction == smoothness)]
        true_n_per_class = annot[(annot.surface == surface) & (annot.smoothness == smoothness)]
        true_n_per_class = true_n_per_class[true_n_per_class.image_id.isin(results.image_id)]
        pred_n = len(pred_annot_class)
        true_n = len(true_n_per_class)
        pred_annot_class = pred_annot_class.set_index("image_id").join(annot[['image_id', 'surface', 'smoothness']].set_index("image_id"), how="left").reset_index()
        pred_annot_class["prediction_int"] = pred_annot_class.prediction.apply(smoothness_to_int)
        pred_annot_class["smoothness_int"] = pred_annot_class.smoothness.apply(smoothness_to_int)

        exclusions_n = len(pred_annot_class[(pred_annot_class.surface == "nostreet")])
        exclusions_perc = round(exclusions_n / pred_n*100, 2)
        tp = len(pred_annot_class[(pred_annot_class.preselection_type == pred_annot_class.surface) & 
                              (pred_annot_class.prediction == pred_annot_class.smoothness)])

        # compute MSE
        pred_annot_class_no_na = pred_annot_class[pred_annot_class.smoothness_int.notna() & pred_annot_class.prediction_int.notna()].copy()
        pred_annot_class_no_na["squared_error"] = abs(pred_annot_class_no_na.smoothness_int - pred_annot_class_no_na.prediction_int).apply(lambda x: x**2)
        rmse = round(root_mean_squared_error(pred_annot_class_no_na.smoothness_int, pred_annot_class_no_na.prediction_int), 2)
        
        counts = pd.concat([counts, 
                            pd.Series([exp_type, exp_id, batch_id, surface, smoothness, total_n, pred_n, true_n,
                                       exclusions_n, exclusions_perc, tp, rmse])], axis=1)

    counts = counts.T
    counts.columns = ["exp_type", "exp", "batch_id", "surface", "smoothness", "total_n", 
                      "gpt_pred_n", "true_n", "exclusions (nostreet)", "excl. %", "tp", "rmse"]
    counts["tp/n_pred"] = (counts.tp / counts.gpt_pred_n * 100).astype(float).round(2)
    counts["recall"] = (counts.tp / counts.true_n * 100).astype(float).round(2)
    #counts["gpt_precision(only valid)"] = (counts.tp / (counts.gpt_pred_n - counts["exclusions (nostreet)"]) * 100).astype(float).round(2)
    counts["hits"] = (counts.tp / counts.total_n * 100).astype(float).round(2)
    return (counts)

In [13]:
class_selection = ["paving_stones", "bad"], ["paving_stones", "intermediate"], ["asphalt", "bad"]
gpt_exp1 = exp_results(1, annotations_folder, gpt_folder, class_selection, exp_type="GPT-4o")
embedding_exp1 = exp_results(1, annotations_folder, embedding_folder, class_selection, exp_type="SimS")

class_selection = ["paving_stones", "bad", 1], ["paving_stones", "bad", 2], ["paving_stones", "intermediate", 1],["paving_stones", "intermediate", 2], ["asphalt", "bad", 1], ["asphalt", "bad", 2]
gpt_exp2 = exp_results(2, annotations_folder, gpt_folder, class_selection, exp_type="GPT-4o")

In [7]:
results = pd.concat([gpt_exp1, gpt_exp2, embedding_exp1], axis=0)
results

,exp_type,exp,batch_id,surface,smoothness,total_n,gpt_pred_n,true_n,exclusions (nostreet),excl. %,tp,rmse,tp/n_pred,recall,hits
0,GPT-4o,1,None,paving_stones,bad,208,17,21,4,23.53,11,0.39,64.71,52.38,5.29
0,GPT-4o,1,None,paving_stones,intermediate,1000,340,165,100,29.41,137,0.63,40.29,83.03,13.70
0,GPT-4o,1,None,asphalt,bad,1014,185,90,45,24.32,73,0.8,39.46,81.11,7.20
0,GPT-4o,2,1,paving_stones,bad,712,54,10,48,88.89,2,0.58,3.70,20.00,0.28
0,GPT-4o,2,2,paving_stones,bad,712,11,10,8,72.73,2,0.63,18.18,20.00,0.28
0,GPT-4o,2,1,paving_stones,intermediate,712,162,57,63,38.89,46,0.73,28.40,80.70,6.46
0,GPT-4o,2,2,paving_stones,intermediate,712,175,57,52,29.71,54,0.78,30.86,94.74,7.58
0,GPT-4o,2,1,asphalt,bad,1998,55,5,39,70.91,5,0.9,9.09,100.00,0.25
0,GPT-4o,2,2,asphalt,bad,1995,33,7,14,42.42,7,1.03,21.21,100.00,0.35
0,SimS,1,None,paving_stones,bad,208,204,21,74,36.27,21,1.69,10.29,100.00,10.10


## Compute OSMT for v101

In [15]:
annot_v101 = pd.read_csv(f"{annotations_folder}/V101.csv")
annot_v101["image_id"] = annot_v101.image.apply(lambda x: str.split(x, "/")[-1]).apply(lambda x: int(str.split(x, ".jpg")[0]))
annot_v101.loc[annot_v101.nostreet.notna(), "surface"] = "nostreet"
annot_v101.loc[annot_v101.nostreet.notna(), "smoothness"] = "nostreet"
# add - yes / no
annot_v101.loc[annot_v101.correct == "no", "surface"] = "false"
annot_v101.loc[annot_v101.correct == "no", "smoothness"] = "false"

annot_v101 = annot_v101[["image_id", "surface", "smoothness"]]
osm_tag_v101 = pd.read_csv("/Users/alexandra/Nextcloud-HTW/SHARED/SurfaceAI/data/mapillary_images/training/V101/metadata/train_image_selection_metadata.csv")
osm_tag_v101 = osm_tag_v101[["id", "surface_clean", "smoothness_clean"]]

annot_v101 = annot_v101.set_index("image_id").join(osm_tag_v101.set_index("id"), how="left")

n = annot_v101.groupby(["surface_clean", "smoothness_clean"]).size()
tp = (annot_v101[(annot_v101.surface == annot_v101.surface_clean) & 
                 (annot_v101.smoothness == annot_v101.smoothness_clean)]
        .groupby(["surface_clean", "smoothness_clean"]).size())

osmt = pd.DataFrame([tp, n]).T
osmt.columns = ["tp", "n"]
osmt["OSMT"] = round((osmt.tp / osmt.n)*100, 2)
osmt.reset_index(inplace=True)

osmt["class"] = osmt.surface_clean + "-" + osmt.smoothness_clean
osmt

,surface_clean,smoothness_clean,tp,n,OSMT,class
0,asphalt,bad,102,1014,10.06,asphalt-bad
1,paving_stones,bad,21,210,10.00,paving_stones-bad
2,paving_stones,intermediate,228,1002,22.75,paving_stones-intermediate


## Table for paper

In [16]:
results

,exp_type,exp,batch_id,surface,smoothness,total_n,gpt_pred_n,true_n,exclusions (nostreet),excl. %,tp,rmse,tp/n_pred,recall,hits,class
0,GPT-4o,1,None,paving_stones,bad,208,17,21,4,23.53,11,0.39,64.71,52.38,5.29,paving_stones-bad
0,GPT-4o,1,None,paving_stones,intermediate,1000,340,165,100,29.41,137,0.63,40.29,83.03,13.70,paving_stones-intermediate
0,GPT-4o,1,None,asphalt,bad,1014,185,90,45,24.32,73,0.8,39.46,81.11,7.20,asphalt-bad
0,GPT-4o,2,1,paving_stones,bad,712,54,10,48,88.89,2,0.58,3.70,20.00,0.28,paving_stones-bad
0,GPT-4o,2,2,paving_stones,bad,712,11,10,8,72.73,2,0.63,18.18,20.00,0.28,paving_stones-bad
0,GPT-4o,2,1,paving_stones,intermediate,712,162,57,63,38.89,46,0.73,28.40,80.70,6.46,paving_stones-intermediate
0,GPT-4o,2,2,paving_stones,intermediate,712,175,57,52,29.71,54,0.78,30.86,94.74,7.58,paving_stones-intermediate
0,GPT-4o,2,1,asphalt,bad,1998,55,5,39,70.91,5,0.9,9.09,100.00,0.25,asphalt-bad
0,GPT-4o,2,2,asphalt,bad,1995,33,7,14,42.42,7,1.03,21.21,100.00,0.35,asphalt-bad
0,SimS,1,None,paving_stones,bad,208,204,21,74,36.27,21,1.69,10.29,100.00,10.10,paving_stones-bad


In [18]:
results["class"] = results.surface + "-" + results.smoothness
paper_table = results[(results.batch_id.isna()) | (results.batch_id == 2)].pivot(
    index = ["class"],
    columns =  ["exp_type", "exp"],
    values = ["tp/n_pred", "recall"])

paper_table.columns = paper_table.columns.droplevel(0)
paper_table.columns = paper_table.columns.droplevel(1)
paper_table.reset_index(inplace=True)
#paper_table.drop("exp_type", inplace=True, axis=1)

paper_table = paper_table.set_index("class").join(osmt[["class", "OSMT"]].set_index("class"))
paper_table = paper_table.reset_index().iloc[:,[0,7,1,4,2,3,6]]
paper_table.columns=(["class", "OSMT", "OSMT+CM+GPT-4o", "", "OSMT+GPT-4o",
                      "SimS+CM+GPT-4o", ""
                      ])

paper_table


,class,OSMT,OSMT+CM+GPT-4o,,OSMT+GPT-4o,SimS+CM+GPT-4o,
0,asphalt-bad,10.06,39.46,81.11,21.21,20.38,47.78
1,paving_stones-bad,10.00,64.71,52.38,18.18,10.29,100.00
2,paving_stones-intermediate,22.75,40.29,83.03,30.86,30.74,47.88


In [11]:
# without experiment 2

results["class"] = results.surface + "-" + results.smoothness
paper_table = results[(results.batch_id.isna()) | (results.batch_id == 2)].pivot(
    index = ["class"],
    columns =  ["exp_type", "exp"],
    values = ["tp/n_pred", "hits"])

paper_table.columns = paper_table.columns.droplevel(0)
paper_table.columns = paper_table.columns.droplevel(1)
paper_table.reset_index(inplace=True)
#paper_table.drop("exp_type", inplace=True, axis=1)

paper_table = paper_table.set_index("class").join(osmt[["class", "OSMT"]].set_index("class"))
paper_table.reset_index(inplace=True)
paper_table["OSM pre-label+type pseudo-label+GPT-4o"] =  paper_table.iloc[:,1].astype(str) + " | " + paper_table.iloc[:,4].astype(str)
paper_table["OSM pre-label+type pseudo-label+SimS"] =  paper_table.iloc[:,3].astype(str) + " | " + paper_table.iloc[:,6].astype(str)

paper_table = paper_table[["class", "OSMT", "OSM pre-label+type pseudo-label+GPT-4o", "OSM pre-label+type pseudo-label+SimS"]]


In [12]:
print(paper_table.to_latex(float_format="%.2f", index=False))

\begin{tabular}{lrll}
\toprule
class & OSMT & OSM pre-label+type pseudo-label+GPT-4o & OSM pre-label+type pseudo-label+SimS \\
\midrule
asphalt-bad & 10.06 & 39.46 | 7.2 & 20.38 | 4.24 \\
paving_stones-bad & 10.00 & 64.71 | 5.29 & 10.29 | 10.1 \\
paving_stones-intermediate & 22.75 & 40.29 | 13.7 & 30.74 | 7.9 \\
\bottomrule
\end{tabular}

